In [82]:

import pandas as pd



# Загрузка данных
df = pd.read_excel('C:/Users/Mi/Downloads/2023-2024 ассесмент база данных.xlsx')
df.columns = df.columns.str.strip()
df_original = df.copy()
# Удаление ненужных столбцов
columns_to_drop = ['ID оценочной сессии', 'Опубликование макета', 'Трудоемкость программы',
                   'Срок реализации программы', 'Целевой уровень развития компетенции',
                   'Дата регистрации', 'Дата начала прохождения оценки']
df.drop(columns=columns_to_drop, inplace=True)

# Преобразуем столбец 'Поток' в два новых столбца
def convert_to_years(year_str):
    start_year = '20' + year_str.split('/')[0]
    end_year = '20' + year_str.split('/')[1]
    return start_year, end_year
df[['Год начала', 'Год окончания']] = df['Поток'].apply(lambda x: pd.Series(convert_to_years(str(x))))

# Преобразуем годы в даты
df['Дата начала'] = pd.to_datetime(df['Год начала'] + '-01-01')
df['Дата окончания'] = pd.to_datetime(df['Год окончания'] + '-01-01')

# Преобразуем числовые столбцы
numeric_cols = ['Результат', 'Количество попыток', 'Время результирующей попытки',
               'Итоговый уровень сформированности компетенций']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# --- ОБЩАЯ СТАТИСТИКА АССЕССМЕНТА ---
# Общие метрики
num_participants = df['ID пользователя'].nunique()
num_graduates = df[df['Статус'] == 'Завершено']['ID пользователя'].nunique()
average_score = df['Результат'].mean()
average_time = df['Время результирующей попытки'].mean()

# Статистика завершения
classification_by_direction = {}
unique_directions = df['Наименование оценочной сессии'].unique()
for direction in unique_directions:
    classification_by_direction[direction] = {
        "Не прошли все 3 этапа": 0,
        "Прошли все 3 этапа": 0,
        "Отчислены": 0
    }
grouped = df.groupby('ID пользователя')
for user_id, user_data in grouped:
    direction = user_data['Наименование оценочной сессии'].iloc[0]
    if "Отчислен" in user_data['Состояние'].values:
        classification_by_direction[direction]["Отчислены"] += 1
    else:
        completed_stages = user_data[user_data['Статус'] == 'Завершено']['Этап оценки'].nunique()
        if completed_stages == 3:
            classification_by_direction[direction]["Прошли все 3 этапа"] += 1
        else:
            classification_by_direction[direction]["Не прошли все 3 этапа"] += 1

# Другие метрики
target_achieved_students = df[df['Итоговый уровень развития компетенции'].isin(['Достигнут', 'Превышен'])]
target_achieved_by_direction = target_achieved_students.groupby('Наименование оценочной сессии')['ID пользователя'].nunique()
listeners_by_direction = df.groupby('Наименование оценочной сессии')['ID пользователя'].nunique()
stage_1_data = df[df['Этап оценки'] == 1]
participants_stage_1_by_direction = stage_1_data.groupby('Наименование оценочной сессии')['ID пользователя'].nunique()

# Вывод результатов первого анализа
print("\nОБЩИЙ АНАЛИЗ")
print(f"\n1. Число участников: {num_participants}")
print(f"2. Число выпускников: {num_graduates}")
print(f"3. Средний балл: {average_score:.2f}")
print(f"4. Среднее время: {average_time:.2f}")

print("\n5. Статистика завершения обучения по направлениям:")
for direction, stats in classification_by_direction.items():
    print(f"\n{direction}:")
    for category, count in stats.items():
        print(f"  {category}: {count}")

print("\n6. Число студентов, достигших целевого показателя:")
print(target_achieved_by_direction.to_string())

print("\n7. Количество слушателей по направлениям:")
print(listeners_by_direction.to_string())

print("\n8. Количество участников на 1 этапе:")
print(participants_stage_1_by_direction.to_string())

# --- УРОВЕНЬ КОМПЕТЕНЦИЙ НА ЭТАПАХ АССЕССМЕНТА ---

# Используем сохраненную копию данных
df = df_original.copy()

# Преобразуем текстовые значения в числовые
level_mapping = {
    'Минимальный исходный': 1,
    'Базовый': 2,
    'Продвинутый': 3,
    'Экспертный': 4
}

# Преобразуем уровни
if df['Итоговый уровень сформированности компетенций'].dtype == 'object':
    df['Итоговый уровень сформированности компетенций'] = (
        df['Итоговый уровень сформированности компетенций']
        .map(level_mapping)
    )

# Удаление ненужных столбцов
columns_to_drop = [
    'ID оценочной сессии', 'Опубликование макета', 'Отраслевая принадлежность',
    'Обучающиеся направления', 'Трудоемкость программы', 'Поток',
    'Срок реализации программы', 'ID пользователя', 'Дата регистрации',
    'Дата начала прохождения оценки', 'Статус', 'Результат',
    'Количество попыток', 'Время результирующей попытки', 'Состояние'
]
df = df.drop(columns=columns_to_drop)

# Удалим строки с пропущенными значениями в уровне компетенций
df = df.dropna(subset=['Итоговый уровень сформированности компетенций'])

# Группировка данных
try:
    grouped = df.groupby(['Этап оценки', 'Наименование оценочной сессии',
                         'Наименование компетенции', 'Итоговый уровень сформированности компетенций']).size().unstack(fill_value=0)

    # Проверим, есть ли данные для анализа
    if not grouped.empty:
        # Расчет процентного соотношения
        percentage_grouped = grouped.div(grouped.sum(axis=1), axis=0) * 100

        # Переименование столбцов
        percentage_grouped = percentage_grouped.rename(columns={
            1: 'Минимальный исходный (%)',
            2: 'Базовый (%)',
            3: 'Продвинутый (%)',
            4: 'Экспертный (%)'
        })

        # Вывод результатов второго анализа
        print("\n\nАНАЛИЗ УРОВНЕЙ СФОРМИРОВАННОСТИ КОМПЕТЕНЦИЙ")
        for (stage, session, competence), data in percentage_grouped.iterrows():
            print(f"\nЭтап оценки: {stage}")
            print(f"Оценочная сессия: {session}")
            print(f"Компетенция: {competence}")
            print("Уровни сформированности компетенций:")
            print(data.to_string())
            print("-" * 50)
    else:
        print("\nНет данных для анализа уровней сформированности компетенций")

except Exception as e:
    print(f"\nОшибка при анализе уровней компетенций: {str(e)}")

# --- СРЕДНИЕ ЗНАЧЕНИЯ ПО НАПРАВЛЕНИЯМ И ГОДАМ ---
print("\n\n АНАЛИЗ СРЕДНИХ ЗНАЧЕНИЙ ")

# Используем сохраненную копию данных
df = df_original.copy()

# Извлекаем год из названия оценочной сессии
df['Год'] = df['Наименование оценочной сессии'].str.extract(r'(\d{4})')

# Альтернативный вариант, если год не извлекается из названия:
if df['Год'].isnull().all():
    df['Год'] = df['Поток'].apply(lambda x: '20' + str(x).split('/')[0] if pd.notnull(x) else None)

# Преобразуем результат в числовой формат
df['Результат'] = pd.to_numeric(df['Результат'], errors='coerce')

# Удаляем строки с пропущенными значениями
df = df.dropna(subset=['Обучающиеся направления', 'Год', 'Результат'])

# Рассчитываем средний балл по направлениям и годам
average_score_by_direction = df.groupby(['Наименование оценочной сессии', 'Год'])['Результат'].mean().reset_index()

# Рассчитываем среднее время по каждому этапу
average_time_by_stage = df.groupby(['Этап оценки', 'Наименование оценочной сессии'])['Время результирующей попытки'].mean().reset_index()

# Среднее количество попыток
average_attempts_by_competency = df.groupby(['Год', 'Наименование оценочной сессии'])['Количество попыток'].mean()

# Рассчитываем средний балл по этапам
average_scores_by_stage = df.groupby(['Год', 'Этап оценки', 'Наименование оценочной сессии'])['Результат'].mean().reset_index()

#Рассчитаем средний балл по компетенциям
average_score_by_competency = df.groupby(['Год', 'Наименование компетенции'])['Результат'].mean()

# Фильтруем данные по первому этапу
first_stage = df[df['Этап оценки'] == 1]

# Рассчитываем средний балл по компетенциям
average_scores_by_competency = first_stage.groupby(['Год', 'Наименование компетенции'])['Результат'].mean().reset_index()

# Вывод результатов
print("\nСредний балл по направлениям и годам:")
print(average_score_by_direction.to_string(index=False))
print("\nСреднее значение времени по каждому этапу:")
print(average_time_by_stage)
print("\nСреднее количество попыток по направлениям и годам:")
print(average_attempts_by_competency)
print("\nСредний балл по этапам и годам:")
print(average_scores_by_stage)
print("\nСредний балл по компетенциям:")
print(average_score_by_competency)
print("\nСредний балл по компетенциям на первом этапе:")
print(average_scores_by_competency)

# --- ВЛИЯНИЕ ВРЕМЕНИ НА РЕЗУЛЬТАТ АССЕССМЕНТА ---
print("\n\n АНАЛИЗ ВЛИЯНИЯ ВРЕМЕНИ НА РЕЗУЛЬТАТЫ ")

# Используем сохраненную копию данных
df = df_original.copy()

# Преобразование данных
df['Время результирующей попытки'] = pd.to_numeric(df['Время результирующей попытки'], errors='coerce')
df['Итоговый уровень сформированности компетенций'] = df['Итоговый уровень сформированности компетенций'].fillna('Не указано')

# Группировка данных по "Наименование оценочной сессии" и "Этап оценки"
grouped_data = df.groupby(['Наименование оценочной сессии', 'Этап оценки'])

# Функция для анализа влияния времени на результаты
def analyze_time_impact(group):
    avg_time = group['Время результирующей попытки'].mean()
    faster_than_avg = group[group['Время результирующей попытки'] < avg_time]
    slower_than_avg = group[group['Время результирующей попытки'] > avg_time]

    faster_results = faster_than_avg['Итоговый уровень сформированности компетенций'].value_counts(normalize=True) * 100
    slower_results = slower_than_avg['Итоговый уровень сформированности компетенций'].value_counts(normalize=True) * 100

    return faster_results, slower_results, avg_time

# Анализ и вывод результатов
for name, group in grouped_data:
    session_name, stage = name
    # Пропускаем группы с недостаточными данными
    if len(group) < 5 or group['Время результирующей попытки'].isnull().all():
        continue

    try:
        faster_results, slower_results, avg_time = analyze_time_impact(group)

        print(f"\nНаименование оценочной сессии: {session_name}, Этап оценки: {stage}")
        print(f"Среднее время прохождения: {avg_time:.2f} минут")
        print("\nРезультаты при прохождении быстрее среднего:")
        print(faster_results.to_string())
        print("\nРезультаты при прохождении дольше среднего:")
        print(slower_results.to_string())
        print("-" * 50)
    except Exception as e:
        print(f"Ошибка при анализе для {session_name}, этап {stage}: {str(e)}")
        continue



ОБЩИЙ АНАЛИЗ

1. Число участников: 1921
2. Число выпускников: 1896
3. Средний балл: 0.37
4. Среднее время: 33.36

5. Статистика завершения обучения по направлениям:

2023 Анализ данных для прикладных областей:
  Не прошли все 3 этапа: 98
  Прошли все 3 этапа: 599
  Отчислены: 5

2023 Современные технологии получения и анализа данных в «Умном городе»:
  Не прошли все 3 этапа: 32
  Прошли все 3 этапа: 280
  Отчислены: 1

2023 Разработчик виртуальной и дополненной реальности:
  Не прошли все 3 этапа: 25
  Прошли все 3 этапа: 174
  Отчислены: 2

2023 Разработка на платформе «1С: Предприятие»:
  Не прошли все 3 этапа: 16
  Прошли все 3 этапа: 119
  Отчислены: 1

2023 Цифровые технологии управления «Умным городом»:
  Не прошли все 3 этапа: 42
  Прошли все 3 этапа: 419
  Отчислены: 1

2023 Аналитика больших данных в строительстве:
  Не прошли все 3 этапа: 25
  Прошли все 3 этапа: 82
  Отчислены: 0

6. Число студентов, достигших целевого показателя:
Наименование оценочной сессии
2023 Анализ д

In [84]:
import pandas as pd
import os
from datetime import datetime

# ... (ваш предыдущий код до создания final_df остается без изменений) ...

# Создаем словарь для сбора всех данных
data_dict = {}

# --- ОБРАБОТКА СТАТИСТИКИ ЗАВЕРШЕНИЯ ОБУЧЕНИЯ ---
for direction, stats in classification_by_direction.items():
    for category, count in stats.items():
        data_dict[f"Статистика/{direction}/{category}"] = count

# --- ОБРАБОТКА ДОСТИЖЕНИЯ ЦЕЛЕВОГО ПОКАЗАТЕЛЯ ---
for direction, count in target_achieved_by_direction.items():
    data_dict[f"Целевые показатели/{direction}/Достигли"] = count

# --- ОБРАБОТКА КОЛИЧЕСТВА СЛУШАТЕЛЕЙ ---
for direction, count in listeners_by_direction.items():
    data_dict[f"Участники/{direction}/Общее количество"] = count

# --- ОБРАБОТКА УЧАСТНИКОВ 1 ЭТАПА ---
for direction, count in participants_stage_1_by_direction.items():
    data_dict[f"Участники/{direction}/1 этап"] = count

# --- ОБРАБОТКА СРЕДНИХ ЗНАЧЕНИЙ ---
if not average_score_by_direction.empty:
    for idx, row in average_score_by_direction.iterrows():
        direction = row['Наименование оценочной сессии']
        year = row['Год']
        data_dict[f"Средние значения/{direction}/Средний балл ({year})"] = row['Результат']

if not average_time_by_stage.empty:
    for idx, row in average_time_by_stage.iterrows():
        direction = row['Наименование оценочной сессии']
        stage = row['Этап оценки']
        data_dict[f"Средние значения/{direction}/Среднее время (этап {stage})"] = row['Время результирующей попытки']

# --- ОБРАБОТКА УРОВНЕЙ КОМПЕТЕНЦИЙ ---
if 'percentage_grouped' in locals():
    for (stage, session, competence), data in percentage_grouped.iterrows():
        data_dict[f"Уровни компетенций/{session}/Этап {stage}/{competence}/Базовый (%)"] = data['Базовый (%)']
        data_dict[f"Уровни компетенций/{session}/Этап {stage}/{competence}/Продвинутый (%)"] = data['Продвинутый (%)']
        data_dict[f"Уровни компетенций/{session}/Этап {stage}/{competence}/Экспертный (%)"] = data['Экспертный (%)']
        data_dict[f"Уровни компетенций/{session}/Этап {stage}/{competence}/Минимальный исходный (%)"] = data['Минимальный исходный (%)']

# --- ОБРАБОТКА ВЛИЯНИЯ ВРЕМЕНИ НА РЕЗУЛЬТАТЫ ---
if 'grouped_data' in locals():
    for name, group in grouped_data:
        session_name, stage = name
        if len(group) < 5 or group['Время результирующей попытки'].isnull().all():
            continue
        
        try:
            faster_results, slower_results, avg_time = analyze_time_impact(group)
            
            # Добавляем среднее время
            data_dict[f"Влияние времени/{session_name}/Этап {stage}/Среднее время"] = avg_time
            
            # Добавляем результаты для быстрых участников по уровням
            for level, percent in faster_results.items():
                level_name = {
                    1: "Минимальный исходный",
                    2: "Базовый",
                    3: "Продвинутый",
                    4: "Экспертный"
                }.get(level, str(level))
                data_dict[f"Влияние времени/{session_name}/Этап {stage}/Быстрее среднего/{level_name}"] = percent
            
            # Добавляем результаты для медленных участников по уровням
            for level, percent in slower_results.items():
                level_name = {
                    1: "Минимальный исходный",
                    2: "Базовый",
                    3: "Продвинутый",
                    4: "Экспертный"
                }.get(level, str(level))
                data_dict[f"Влияние времени/{session_name}/Этап {stage}/Медленнее среднего/{level_name}"] = percent
                
        except Exception as e:
            continue
# --- ОБРАБОТКА СРЕДНИХ ЗНАЧЕНИЙ ПО КОМПЕТЕНЦИЯМ ---

# Средний балл по компетенциям (все этапы вместе)
if not average_score_by_competency.empty:
    for (year, competence), score in average_score_by_competency.items():
        data_dict[f"Средние значения по компетенциям/Все этапы/{year}/{competence}"] = score

# Средний балл по компетенциям по этапам
if not average_scores_by_competency_stage.empty:
    for idx, row in average_scores_by_competency_stage.iterrows():
        year = row['Год']
        stage = row['Этап оценки']
        competence = row['Наименование компетенции']
        score = row['Результат']
        data_dict[f"Средние значения по компетенциям/Этап {stage}/{year}/{competence}"] = score

# --- ОБРАБОТКА СРЕДНИХ ЗНАЧЕНИЙ ПО ЭТАПАМ ---
if not average_scores_by_stage.empty:
    for idx, row in average_scores_by_stage.iterrows():
        year = row['Год']
        stage = row['Этап оценки']
        direction = row['Наименование оценочной сессии']
        score = row['Результат']
        data_dict[f"Средние значения по этапам/{direction}/Этап {stage} ({year})"] = score

# --- СОЗДАЕМ ИТОГОВЫЙ DATAFRAME ---
final_df = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Значение'])

# --- ФОРМАТИРУЕМ ВЫВОД ---
final_df.index = final_df.index.str.split('/').map(lambda x: " | ".join(x))

# --- ВЫВОД И СОХРАНЕНИЕ ---
print("\nИТОГОВАЯ СВОДНАЯ ТАБЛИЦА:")
print(final_df.sort_index())

try:
    # Создаем уникальное имя файла
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"итоговый_анализ_{timestamp}.xlsx"
    
    # Пробуем сохранить на рабочий стол
    desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
    save_path = os.path.join(desktop_path, output_file)
    
    # Создаем директорию если нужно
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    # Сохраняем файл с красивыми заголовками
    final_df.reset_index().rename(columns={'index': 'Метрика'}).to_excel(
        save_path, 
        index=False,
        sheet_name='Результаты асессмента'
    )
    print(f"\nФайл успешно сохранен: {save_path}")
    
except PermissionError:
    print("\nОшибка: Нет доступа к файлу. Закройте Excel и попробуйте снова.")
except Exception as e:
    print(f"\nОшибка при сохранении: {str(e)}")
    print("Попробуйте сохранить вручную:")
    print(final_df)


ИТОГОВАЯ СВОДНАЯ ТАБЛИЦА:
                                                      Значение
Влияние времени | 2023 Анализ данных для прикла...   31.325301
Влияние времени | 2023 Анализ данных для прикла...   62.650602
Влияние времени | 2023 Анализ данных для прикла...    5.903614
Влияние времени | 2023 Анализ данных для прикла...    0.120482
Влияние времени | 2023 Анализ данных для прикла...   43.462898
...                                                        ...
Целевые показатели | 2023 Аналитика больших дан...  109.000000
Целевые показатели | 2023 Разработка на платфор...  134.000000
Целевые показатели | 2023 Разработчик виртуальн...  190.000000
Целевые показатели | 2023 Современные технологи...  293.000000
Целевые показатели | 2023 Цифровые технологии у...  438.000000

[437 rows x 1 columns]

Файл успешно сохранен: C:\Users\Mi\Desktop\итоговый_анализ_20250405_113445.xlsx
